In [2]:
import sys
# import sklearn
import numpy as np
import pandas as pd
from time import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model

f"Python: {sys.version}"

'Python: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]'

### Setup

In [10]:
trainData = pd.read_csv("tcdml1920-rec-click-pred--training.csv", index_col='recommendation_set_id', low_memory=False)
testData = pd.read_csv("tcdml1920-rec-click-pred--test.csv", index_col='recommendation_set_id', low_memory=False)

#drops 1647 rows with nothing
testData.dropna(how="all", inplace=True)

In [5]:
trainData["set_clicked"].value_counts()

0    378924
1      6763
Name: set_clicked, dtype: int64

In [14]:
trainData.columns

Index(['user_id', 'session_id', 'query_identifier', 'query_word_count',
       'query_char_count', 'query_detected_language', 'query_document_id',
       'document_language_provided', 'year_published', 'number_of_authors',
       'abstract_word_count', 'abstract_char_count',
       'abstract_detected_language', 'first_author_id',
       'num_pubs_by_first_author', 'organization_id', 'application_type',
       'item_type', 'request_received', 'hour_request_received',
       'response_delivered', 'rec_processing_time', 'app_version', 'app_lang',
       'user_os', 'user_os_version', 'user_java_version', 'user_timezone',
       'country_by_ip', 'timezone_by_ip', 'local_time_of_request',
       'local_hour_of_request', 'number_of_recs_in_set',
       'recommendation_algorithm_id_used', 'algorithm_class', 'cbf_parser',
       'search_title', 'search_keywords', 'search_abstract',
       'time_recs_recieved', 'time_recs_displayed', 'time_recs_viewed',
       'clicks', 'ctr', 'set_clicked'],
  

In [13]:
testData.dropna(axis=1, how='all').shape

(9145, 44)

In [16]:
testData["application_type"].value_counts()

digital_library    5174
e-commerce         3027
blog                938
0                     6
Name: application_type, dtype: int64

In [17]:
trainData["application_type"].value_counts()

digital_library    270145
e-commerce         100070
blog                15212
0                     260
Name: application_type, dtype: int64

In [6]:
trainData["ctr"].value_counts()

0.0000    378924
0.1429      3858
0.3333       977
0.2857       944
0.4286       328
0.2000       144
0.5714       132
1.0000        77
0.6667        70
0.7143        48
0.4000        37
0.8571        33
0.1667        30
0.5000        28
0.2500        12
0.6000         8
1.2000         6
1.2857         5
1.1429         4
1.4286         3
0.7500         3
1.3333         3
1.5714         3
2.0000         3
0.8000         2
2.7143         1
2.5000         1
2.5714         1
1.6667         1
1.6000         1
Name: ctr, dtype: int64

In [36]:
# Row Duplication check TrainData
duplicateRowsTrain = trainData[trainData.duplicated()]
duplicateRowsTrain.shape

(285, 45)

In [13]:
# Row Duplication check TestData
duplicateRowsTest = testData[testData.duplicated()]
duplicateRowsTest.shape

(0, 45)

In [11]:
trainData = trainData.drop_duplicates()

In [12]:
trainData.replace(["\\N", "nA", "Not provided", "unknown", "*unknown*"], np.nan, inplace=True)
testData.replace(["\\N", "nA", "Not provided",  "unknown", "*unknown*"], np.nan, inplace=True)

In [29]:
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
trainData.sample(5)

,user_id,session_id,query_identifier,query_word_count,query_char_count,query_detected_language,query_document_id,document_language_provided,year_published,number_of_authors,abstract_word_count,abstract_char_count,abstract_detected_language,first_author_id,num_pubs_by_first_author,organization_id,application_type,item_type,request_received,hour_request_received,response_delivered,rec_processing_time,app_version,app_lang,user_os,user_os_version,user_java_version,user_timezone,country_by_ip,timezone_by_ip,local_time_of_request,local_hour_of_request,number_of_recs_in_set,recommendation_algorithm_id_used,algorithm_class,cbf_parser,search_title,search_keywords,search_abstract,time_recs_recieved,time_recs_displayed,time_recs_viewed,clicks,ctr,set_clicked
recommendation_set_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
82555,NaN,NaN,Withheld for privacy,10,62,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,digital_library,academic_publication,14/01/2019 11:35,11,14/01/2019 11:35,8.529710,4.3.1,en,NaN,NaN,NaN,NaN,FI,200,14/01/2019 12:35,12,7,NaN,NaN,NaN,no,no,no,NaN,NaN,NaN,0,0.0,0
271003,NaN,NaN,Asus Laptop X51L Compatible Power Supply Plug ...,9,53,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,e-commerce,NaN,02/07/2019 08:25,8,02/07/2019 08:25,0.048823,NaN,en,NaN,NaN,NaN,NaN,DE,NaN,NaN,NaN,3,12,content_based_filtering,edismax_QP,yes,yes,yes,NaN,NaN,NaN,0,0.0,0
60200,NaN,NaN,Withheld for privacy,12,78,pt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,digital_library,academic_publication,17/12/2018 19:07,19,17/12/2018 19:07,2.321520,4.3.1,en,NaN,NaN,NaN,NaN,BR,-200,17/12/2018 16:07,16,7,1,content_based_filtering,standard_QP,yes,no,no,NaN,NaN,NaN,0,0.0,0
161059,NaN,NaN,Withheld for privacy,12,85,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,digital_library,academic_publication,25/03/2019 16:15,16,25/03/2019 16:15,3.036940,4.3.1,en,NaN,NaN,NaN,NaN,DE,100,25/03/2019 16:15,16,7,2,content_based_filtering,edismax_QP,yes,yes,no,NaN,NaN,NaN,0,0.0,0
63991,NaN,NaN,Withheld for privacy,5,49,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,digital_library,academic_publication,21/12/2018 13:40,13,21/12/2018 13:40,0.084040,4.3.1,zh,NaN,NaN,NaN,NaN,CN,800,21/12/2018 20:40,20,7,12,content_based_filtering,edismax_QP,yes,yes,yes,NaN,NaN,NaN,0,0.0,0


In [ ]:
df[df['Sales'] < s]

In [24]:
trainData.iloc[:, :-1].values[0]

array([nan, nan, 'Withheld for privacy', '6', '35', 'en', nan, nan, nan,
       nan, nan, nan, nan, nan, nan, 1, 'digital_library',
       'academic_publication', '03/12/2018 16:28', 16, '03/12/2018 16:28',
       0.681027, '4.3.1', 'en', nan, nan, nan, nan, 'NL',
       'Europe/Amsterdam', nan, nan, 7, '1', 'content_based_filtering',
       'standard_QP', 'yes', 'no', 'no', nan, nan, nan, 0, 0.0],
      dtype=object)

In [22]:
threshold = 0.30

#Dropping rows with missing value rate higher than threshold
trainData.loc[trainData.isnull().mean(axis=1) < threshold].shape

(103996, 45)

In [30]:
trainData.columns

Index(['user_id', 'session_id', 'query_identifier', 'query_word_count',
       'query_char_count', 'query_detected_language', 'query_document_id',
       'document_language_provided', 'year_published', 'number_of_authors',
       'abstract_word_count', 'abstract_char_count',
       'abstract_detected_language', 'first_author_id',
       'num_pubs_by_first_author', 'organization_id', 'application_type',
       'item_type', 'request_received', 'hour_request_received',
       'response_delivered', 'rec_processing_time', 'app_version', 'app_lang',
       'user_os', 'user_os_version', 'user_java_version', 'user_timezone',
       'country_by_ip', 'timezone_by_ip', 'local_time_of_request',
       'local_hour_of_request', 'number_of_recs_in_set',
       'recommendation_algorithm_id_used', 'algorithm_class', 'cbf_parser',
       'search_title', 'search_keywords', 'search_abstract',
       'time_recs_recieved', 'time_recs_displayed', 'time_recs_viewed',
       'clicks', 'ctr', 'set_clicked'],
  

In [42]:
testData["application_type"].value_counts(normalize = False, dropna = False)

digital_library    5174
e-commerce         3027
blog                938
0                     6
Name: application_type, dtype: int64

In [47]:
testData.groupby("search_keywords").mean()

,organization_id,hour_request_received,set_clicked
search_keywords,,,
no,2.747332,12.836220,NaN
yes,2.656268,12.811953,NaN


In [38]:
trainData["application_type"].value_counts(normalize = False, dropna = False)

digital_library    270145
e-commerce         100070
blog                15212
0                     260
Name: application_type, dtype: int64

In [ ]:
# testData[testData['abstract_detected_language'].isnull()]

In [ ]:
testData.isnull().sum().sort_values()

In [ ]:
numeric_columns = [
    "query_word_count",
    "query_char_count",
    "query_document_id",
    "year_published",
    "number_of_authors",
    "abstract_char_count",
    "abstract_word_count",
    "first_author_id",
    "num_pubs_by_first_author",
    "hour_request_received",
    "local_hour_of_request",
    "recommendation_algorithm_id_used",
    "clicks",
]

In [ ]:
time_columns = [
    "request_received",
    "response_delivered",
    "local_time_of_request",
    "time_recs_recieved",
    "time_recs_displayed",
    "time_recs_viewed",
]

In [ ]:
for col in numeric_columns:
    trainData[col] = pd.to_numeric(trainData[col], errors="coerce")
    testData[col] = pd.to_numeric(testData[col], errors="coerce")

In [ ]:
for col in time_columns:
    trainData[col] = pd.to_datetime(trainData[col], format="%d/%m/%Y %H:%M")
    testData[col] = pd.to_datetime(testData[col], format="%d/%m/%Y %H:%M")

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5)
trainData["query_document_id"].value_counts()

In [ ]:
trainData.columns

In [ ]:
trainData.dtypes

In [ ]:
trainData['set_clicked'].value_counts().plot(kind='pie', figsize=(8,8))

In [ ]:
trainData.corr().style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
f = plt.figure(figsize=(19, 15))
plt.matshow(trainData.corr(), fignum=f.number)
plt.xticks(range(trainData.shape[1]), trainData.columns, fontsize=14, rotation=45)
plt.yticks(range(trainData.shape[1]), trainData.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

285 Duplicated train rows

In [ ]:
# Row Duplication check TestData
duplicateRowsTest = testData[testData.duplicated()]
duplicateRowsTest.shape

### Basic visualizations

In [ ]:
plt.figure(figsize=(24, 4))
# sns.countplot(x="query_identifier", data=train_set)
# plt.show()
sns.countplot(x="document_language_provided", data=train_set)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="abstract_detected_language", data=train_set)
plt.show()
sns.countplot(x="application_type", data=train_set)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="item_type", data=train_set)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="app_version", data=train_set, order=pd.value_counts(
    train_set['app_version']).iloc[:10].index)
plt.show()
sns.countplot(x="app_lang", data=train_set)
plt.show()
sns.countplot(x="user_os", data=train_set)
plt.show()
# sns.countplot(x="user_os_version", data=train_set)
# plt.show()
# sns.countplot(x="user_java_version", data=train_set)
# plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="user_timezone", data=train_set,
              order=pd.value_counts(train_set['user_timezone']).index)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="country_by_ip", data=train_set, order=pd.value_counts(
    train_set['country_by_ip']).iloc[:15].index)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="timezone_by_ip", data=train_set, order=pd.value_counts(
    train_set['timezone_by_ip']).iloc[:15].index)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="algorithm_class", data=train_set)
plt.show()

### Encoding
- One hot
- Target

- Smote 
- Giving more weight to yes